Action1	Santa的接待安排
圣诞节前100天，Santa开放了workshop，欢迎以家庭单位的参观，如何更合理的安排这些家庭参观？
每个家庭有10个选择choice0-9，数字代表了距离圣诞节的天数，比如 1代表12月24日，每个家庭必须并且只安排一次参观
家庭数量 5000，即family_id 为[0, 4999]，每天访问的人数需要在125-300人

为了更合理的计算Santa的安排能力，我们使用preference cost和accounting penalty两个指标
1）preference cost，代表Santa的个性化安排能力
2）accounting penalty，代表Santa安排的财务成本
每天接待的人员数N(d)如果大于125，就会拥挤，产生过多的清洁成本
最终的 Score = preference cost + accounting penalty
最终提交每个家庭的安排 submission.csv


1、使用LP对大部分家庭进行安排（30points）
2、使用MIP对剩余家庭进行安排（30points）
3、对得到的解决方案进行优化（20points）

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Step1, 数据加载
data=pd.read_csv('./L22/santa/family_data.csv',index_col='family_id')
data

choice_0  choice_1  choice_2  choice_3  choice_4  choice_5  \
family_id                                                               
0                52        38        12        82        33        75   
1                26         4        82         5        11        47   
2               100        54        25        12        27        82   
3                 2        95         1        96        32         6   
4                53         1        47        93        26         3   
...             ...       ...       ...       ...       ...       ...   
4995             16         1        66        33        18        70   
4996             88        66        20        17        26        54   
4997             32        66        54        17        27        21   
4998             67        92         4        17        53        77   
4999             13        11        25        80        88        40   

           choice_6  choice_7  choice_8  choice_9  n_people  
family_id                                                    
0                64        76        10        28         4  
1                38         6        66        61         4  
2                10        89        80        33         3  
3                40        31         9        59         2  
4                46        16        42        39         4  
...             ...       ...       ...       ...       ...  
4995             56        46        86        60         4  
4996             81        91        59        48         2  
4997             74        81         3         7         6  
4998              1        12        26        70         5  
4999             96        39        18        47         4  

[5000 rows x 11 columns]

## Step2，数据预处理
* 1）计算Perference Cost矩阵 pcost_mat
* 2）计算Accounting Cost矩阵 acost_mat
* 3）计算每个家庭的人数 FAMILY_SIZE
* 4）每个家庭的倾向选择（choice_） DESIRED

In [3]:
#n代表家庭成员个数,如果满足choice需求,需要的penalty
def get_penalty(n,choice):
    penalty=None
    if choice == 0:
        penalty = 0
    if choice == 1:
        penalty = 50
    if choice == 2:
        penalty = 50 + 9 * n
    if choice == 3:
        penalty = 100 + 9* n
    if choice == 4:
        penalty = 200 + 9 * n
    if choice == 5:
        penalty = 200 + 18 * n
    if choice == 6:
        penalty = 300 + 18 * n
    if choice == 7:
        penalty = 300 + 36 * n
    if choice == 8:
        penalty = 400 + 36 * n
    if choice == 9:
        penalty = 500 + (36 + 199) * n
    if choice >9:
        penalty = 500 + (36 + 398) * n
    return penalty

In [4]:
N_DAYS = 100 #安排的天数
N_FAMILY = 5000  #家庭ID的个数
#承载量
MIN_OCCUPANCY =125
MAX_OCCUPANCY =300

#计算pcost_mat,每个家庭在什么时候(day 0-99)访问时的penalty
pcost_mat = np.full(shape = (N_FAMILY,100),fill_value = 999999)
for f in range(N_FAMILY):
    #家庭成员数
    f_num =data.loc[f,'n_people']
    #print(f_num)
    #对于第f个家庭,初始化pcost = other choice下的penalty
    pcost_mat[f,:] = get_penalty(f_num,10)
    #计算choice 0-9的penalty
    for choice in range(10):
        temp = data.loc[f][choice]#choice的天数
        penalty = get_penalty(f_num,choice)
        pcost_mat[f,temp-1] = penalty
pcost_mat    

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [5]:
#家庭成员数:f_num = 4(4比较多),other choice = 2236
pcost_mat.shape

(5000, 100)

In [6]:
%%time
#计算accounting penalty矩阵
#前一天的参观人数,当一天的参观人数
acost_mat = np.zeros(shape = (MAX_OCCUPANCY+1,MAX_OCCUPANCY+1),dtype=np.float64)#300*300,最大参观人数/125*300
for i in range(acost_mat.shape[0]):#当天安排的人数
    for j in range(acost_mat.shape[1]):#前一天安排的人数
        diff = abs(i-j)#绝对值
        acost_mat[i,j] = max(0,(i - 125) / 400 * i ** (0.5 + diff / 50.0))

acost_mat

Wall time: 250 ms


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

In [7]:
#计算每个家庭的人数 FAMILY_SIZE
FAMILY_SIZE = data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4], dtype=int64)

In [8]:
#每个家庭的倾向选择（choice_）DESIRED(代表每个家庭choice-1)
#pcost_mat的day是1-100  对应的下标是0-99
DESIRED = data.values[:,:-1]-1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]], dtype=int64)

# Step3，使用LP和MIP求解 规划方案
#1）先使用LP 对绝大部分家庭进行规划 
#2）再使用MIP 对剩余家庭进行规划 
#3）汇总两边的结果 => 最终规划方案 

In [9]:
#1）先使用LP 对绝大部分家庭进行规划:线性规划
from ortools.linear_solver import pywraplp

def solveLP():
    #优化器
    solver= pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    #solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    #定义变量
    x ={}  #family_id在第j天是否参观
    #每一天有那些家庭
    candidates = [[] for x in range(N_DAYS)]
    
    for i in range(N_FAMILY):
        for j in DESIRED[i,:]:#family_id的choice
            candidates[j].append(i)#在第j天有第i个family参观
            #定义决策变量x[i,j]  i代表family_id,j代表第j天参观
            x[i,j] = solver.BoolVar('x[%i,%i]'%(i,j))
    #约束条件
    #100天每天参观的人数 x[i,j]=0或1
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]#j代表1-100天 
    #每个家庭在10个choice中出现的总数
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) for i in range(N_FAMILY)]
    
    #定义目标函数preference cost部分
    preference_cost = solver.Sum([pcost_mat[i,j]*x[i,j]for i in range(N_FAMILY) for j in DESIRED[i,:]])
    
    #满足preference cost最小
    solver.Minimize(preference_cost)
    
    #人为增加约束条件constraints
    for j in  range(N_DAYS-1):
        #当前人数不超过前一天人数+25
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1] <= 25) #当天-前一天<=25
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j] <= 25)
    #每个家庭都在10个choice里面出现一次
    for i in range(N_FAMILY):
        solver.Add(family_presence[i]==1)
    #每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >=MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j] <=MAX_OCCUPANCY)
    result = solver.Solve()   
    temp = [(i,j,x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i,:] if x[i,j].solution_value()>0]
    #print(temp)
    print(solver.Objective().Value())
    #得到参观日期的安排
    df = pd.DataFrame(temp,columns =['family_id','day','result'])
    return df

In [10]:
%%time
result = solveLP()

73702.31696428571
Wall time: 32.6 s


In [11]:
result

family_id  day  result
0             0   51     1.0
1             1   25     1.0
2             2   99     1.0
3             3    1     1.0
4             4   52     1.0
...         ...  ...     ...
5068       4995   15     1.0
5069       4996   87     1.0
5070       4997   31     1.0
5071       4998   91     1.0
5072       4999   12     1.0

[5073 rows x 3 columns]

In [12]:
result['result'].value_counts()

1.000000    4301
1.000000     627
0.750000       2
0.666667       2
0.666667       2
            ... 
0.750000       1
0.400000       1
0.400000       1
0.400000       1
0.947917       1
Name: result, Length: 144, dtype: int64

In [13]:
#设置阈值
THRS = 0.999
#已经安排上的结果
assigned_df = result[result.result > THRS]
assigned_df

family_id  day  result
0             0   51     1.0
1             1   25     1.0
2             2   99     1.0
3             3    1     1.0
4             4   52     1.0
...         ...  ...     ...
5068       4995   15     1.0
5069       4996   87     1.0
5070       4997   31     1.0
5071       4998   91     1.0
5072       4999   12     1.0

[4931 rows x 3 columns]

In [14]:
#没有安排的,不为0和1
unassigned_df = result[(result.result < THRS) & (result.result > 1-THRS)]
unassigned_df

family_id  day  result
59           59   38    0.25
60           59   14    0.75
241         240   32    0.75
242         240   56    0.25
264         262   31    0.50
...         ...  ...     ...
4983       4912    8    0.40
4985       4914   38    0.60
4986       4914   43    0.40
5033       4961   53    0.75
5034       4961   15    0.25

[138 rows x 3 columns]

In [15]:
#计算每天访问的人数(根据 assigned_df)
assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]
assigned_df

<ipython-input-15-41d6f9ce691f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]


family_id  day  result  family_size
0             0   51     1.0            4
1             1   25     1.0            4
2             2   99     1.0            3
3             3    1     1.0            2
4             4   52     1.0            4
...         ...  ...     ...          ...
5068       4995   15     1.0            4
5069       4996   87     1.0            2
5070       4997   31     1.0            6
5071       4998   91     1.0            5
5072       4999   12     1.0            4

[4931 rows x 4 columns]

In [16]:
#按照day进行聚合
occupancy = assigned_df.groupby('day').family_size.sum().values
occupancy

array([290, 271, 294, 293, 263, 242, 223, 247, 273, 297, 288, 292, 275,
       250, 238, 272, 292, 292, 271, 248, 223, 244, 264, 291, 292, 296,
       273, 249, 234, 251, 278, 283, 252, 235, 205, 184, 202, 233, 253,
       231, 210, 183, 204, 229, 247, 281, 256, 223, 204, 198, 222, 248,
       255, 223, 208, 185, 173, 196, 219, 198, 174, 141, 124, 121, 149,
       170, 178, 158, 136, 123, 125, 135, 158, 185, 158, 135, 121, 125,
       142, 167, 186, 167, 138, 120, 128, 155, 174, 203, 177, 152, 130,
       122, 130, 158, 178, 158, 128, 125, 122, 124], dtype=int64)

In [17]:
min_occupancy = np.array([max(0,MIN_OCCUPANCY - x) for x in occupancy])
min_occupancy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 1], dtype=int64)

In [18]:
max_occupancy = np.array([MAX_OCCUPANCY - x for x in occupancy])
max_occupancy

array([ 10,  29,   6,   7,  37,  58,  77,  53,  27,   3,  12,   8,  25,
        50,  62,  28,   8,   8,  29,  52,  77,  56,  36,   9,   8,   4,
        27,  51,  66,  49,  22,  17,  48,  65,  95, 116,  98,  67,  47,
        69,  90, 117,  96,  71,  53,  19,  44,  77,  96, 102,  78,  52,
        45,  77,  92, 115, 127, 104,  81, 102, 126, 159, 176, 179, 151,
       130, 122, 142, 164, 177, 175, 165, 142, 115, 142, 165, 179, 175,
       158, 133, 114, 133, 162, 180, 172, 145, 126,  97, 123, 148, 170,
       178, 170, 142, 122, 142, 172, 175, 178, 176], dtype=int64)

In [19]:
#2）再使用MIP 对剩余家庭进行规划
def solveMIP(familes,min_occupancy,max_occupancy):
    #创建求解器
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    #需要安排的家庭
    n_familes = len(familes)
    x={} #family_id 在第J天否参观
    #每一天有哪些家庭
    candidates = [[] for x in range(N_DAYS)]

    for i in familes:
        for j in DESIRED[i,:]:#family_id的choice
            candidates[j].append(i)#在第j天有第i个family参观
            #定义决策变量x[i,j]  i代表family_id,j代表第j天参观
            x[i,j] = solver.BoolVar('x[%i,%i]'%(i,j))
    
    #约束条件
    #100天每天参观的人数 x[i,j]=0或1
    daily_occupancy = [solver.Sum([x[i,j]*FAMILY_SIZE[i] for i in candidates[j]])\
                       for j in range(N_DAYS)]#j代表1-100天 
    #每个家庭在10个choice中出现的总数
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) \
                       for i in familes]
    
    #定义目标函数preference cost部分
    preference_cost = solver.Sum([pcost_mat[i,j]*x[i,j]for i in familes \
                                  for j in DESIRED[i,:]])
    
    #满足preference cost最小
    solver.Minimize(preference_cost)
    
    #每个家庭都在10个choice里面出现一次
    for i in range(n_familes):
        solver.Add(family_presence[i]==1)
    #每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >=min_occupancy[j])
        solver.Add(daily_occupancy[j] <=max_occupancy[j])
    result = solver.Solve()   
    temp = [(i,j) for i in familes for j in DESIRED[i,:] if x[i,j].solution_value()>0]
    #计算剩余家庭的去安排
    df = pd.DataFrame(temp,columns =['family_id','day'])
    return df

In [20]:
%%time
#没有安排的family_id
unassigned = unassigned_df.family_id.unique()
result = solveMIP(unassigned,min_occupancy,max_occupancy)
result

Wall time: 269 ms


family_id  day
0          59   38
1         240   32
2         262   31
3         357   24
4         488   39
..        ...  ...
64       4869   59
65       4886   98
66       4912   17
67       4914   38
68       4961   53

[69 rows x 2 columns]

In [21]:
unassigned = unassigned_df.family_id.unique()
unassigned

array([  59,  240,  262,  357,  488,  610,  630,  724,  926,  941,  961,
       1012, 1106, 1167, 1176, 1218, 1235, 1271, 1287, 1387, 1444, 1484,
       1536, 1610, 1611, 1653, 1720, 1791, 1977, 2004, 2101, 2135, 2229,
       2318, 2344, 2395, 2641, 2778, 2862, 2874, 3033, 3054, 3104, 3118,
       3228, 3278, 3280, 3348, 3470, 3504, 3642, 3649, 3786, 4045, 4077,
       4084, 4404, 4421, 4431, 4471, 4518, 4661, 4820, 4850, 4869, 4886,
       4912, 4914, 4961], dtype=int64)

In [22]:
#3）汇总两边的结果 => 最终规划方案
df = pd.concat((assigned_df[['family_id','day']],result)).sort_values('family_id')
df

family_id  day
0             0   51
1             1   25
2             2   99
3             3    1
4             4   52
...         ...  ...
5068       4995   15
5069       4996   87
5070       4997   31
5071       4998   91
5072       4999   12

[5000 rows x 2 columns]

In [23]:
from numba import njit
#Step4, 结果评估按照evaluation标准，计算Score = preference cost + accounting penalty
#根据安排情况计算这个安排的preference cost
@njit(fastmath=True)
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1,dtype=np.int64)
    penalty = 0
    for (i,p) in enumerate(prediction):
        #计算家庭人数
        n = FAMILY_SIZE[i]
        #第i个家庭,P天访问时的cost
        penalty += pcost_mat[i,p]
        #计算当天的人数
        daily_occupancy[p] += n
    return penalty,daily_occupancy

#根据安排情况计算这个安排的accounting cost
@njit(fastmath=True)
#@njit()
def acost(daily_occupancy):
    num_out_of_range = 0
    accounting_cost = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    
    for day in range(N_DAYS):
        n_pl = daily_occupancy[day+1] #前一天
        n = daily_occupancy[day] #当天
        #print(n,n_pl)
        #如果超过了承载该范围,则设置out of range
        num_out_of_range += (n>MAX_OCCUPANCY) or (n<MIN_OCCUPANCY)
        #计算accounting cost
        accounting_cost += acost_mat[n,n_pl]
    return accounting_cost,num_out_of_range

#根据安排prediction
@njit(fastmath=True)
def cost_function(prediction):
    #基于prediction,计算perference cost和accounting cost
    penalty,daily_occupancy = pcost(prediction) #统计perference cost和每天承载数量
    accounting_cost,num_out_of_range = acost(daily_occupancy) #根据每一天承载数量,计算accounting cost
    final_score = penalty + accounting_cost + num_out_of_range * 99999999
    return final_score

In [24]:
prediction = df.day.values
prediction

array([51, 25, 99, ..., 31, 91, 12], dtype=int64)

In [25]:
print(cost_function(prediction))

77544.53130943188


In [26]:
acost_mat.shape

(301, 301)

In [27]:
def save(pred,filename):
    result = pd.DataFrame(range(N_FAMILY),columns=['family_id'])
    result['assigned_day'] = pred + 1
    result.to_csv(filename,index = False)
    print(filename + ' saved')
    return result
result = save(prediction,'submissionl.csv')

submissionl.csv saved


In [28]:
#Step5，方案优化
#通过更换family_id的选择，查找更好的score,每次更换后，都对方案进行评估，选择更小的score方案

In [29]:
def find_better(pred):
    fobs = np.argsort(FAMILY_SIZE)
    
    #print(fobs)
    #print(np.sort(FAMILY_SIZE))
    score = cost_function(pred)
    original_score = np.inf#打擂法
    
    #如果找不到更新,则退出
    while score<original_score:
        original_score = score
        
        for family_id in fobs:
            for pick in range(10):
                #得到family_id的日期day
                day = DESIRED[family_id,pick]
                #该family的原有日期 oldvalue
                oldvalue =pred[family_id]
                #将oldvalue 替换为现在的choice pick
                pred[family_id] = day
                #重新计算分数
                new_score = cost_function(pred)
                #如果比原来小,更新choice成功
                if new_score<score:
                    score = new_score
                else:#设置为原来的oldvalue
                    pred[family_id] = oldvalue
                
        print(score,end = '\r')
    print(score)

In [30]:
new =prediction.copy()
find_better(new)

73926.62920526078


In [31]:
cost_function((new))

73926.62920526078

In [32]:
result = save(new,'submission2.csv')

submission2.csv saved
